# SQL Introduction

We will learn how to use SQL today (mostly by example)

Our example data comes from the [Instacart dataset](./01_data_description.ipynb) we discussed previously

In [1]:
import os
import pandas as pd
import sqlalchemy as sa


from sqlalchemy.ext.declarative import declarative_base

## SQL (structured query language)

SQL is a "query language" that can be used to communicate with (relational) databases.

SQL itself is more of a standard for a language to communicate with databases rather than an implemented programming language which means that each database creates their own implementation of how SQL commands get translated into queries.

**What problem does SQL solve?**

1. Straightforward way to ingest data from a database
2. Industry standard to make database code/requirements (nearly) compatible
3. The implementations often provide great ways to provide multiple levels of "access" to a dataset
  - Some users will be "data users" and will use the data in their projects -- These users can get away with "read only access" to the database
  - Other users will be "data creators" and will maintain and update the data stored in the database -- These users will need to be able to either add data or participate through other administration roles
4. Allows administrators to impose strict requirements across the data -- For example, could impose a uniqueness constraint if we did not want an email to correspond to more than one user etc...

**Our focus today**

Our main focus for this class will be on introducing how to be "database users" rather than "database administrators"

## SQL and SQLAlchemy

We'll now discuss a few details of SQL and SQLAlchemy:

`sqlalchemy` is a Python package that allows one to generically interface with many different "flavors" of SQL (PostgreSQL, MySQL, SQLite, etc...) using Python code.

We will only discuss it briefly today because it isn't the focus of this lecture.

### SQL Tables and Types

As we mentioned, one of the benefits of SQL is that it allows those who are creating the databases to impose tight requirements on what is contained in the data:

* **Tables**: SQL allows one to specify a table with pre-defined columns, cross-table restrictions, and more
* **Types**: Each column in a SQL table must have a specified type. These types are mostly the "usual suspects"
  - Boolean
  - Date
  - Numeric (Float, Integer, ...)
  - String

### Declaring table structures


In [2]:
Base = declarative_base()


class Aisles(Base):
    __tablename__ = "aisles"
    aisle_id = sa.Column(sa.Integer, primary_key=True)
    aisle = sa.Column(sa.String)
    # Aisle_id | Aisle
    # 1        | Aisle of baking goods


class Departments(Base):
    __tablename__ = "departments"
    department_id = sa.Column(sa.Integer, primary_key=True)
    department = sa.Column(sa.String)
    # Department_id | Department
    # 3             | Dry goods


class Products(Base):
    __tablename__ = "products"
    product_id = sa.Column(sa.Integer, primary_key=True)
    product_name = sa.Column(sa.String)
    aisle_id = sa.Column(sa.Integer)  # One can set these to reference the aisles/departments tables
    department_id = sa.Column(sa.Integer)
    # product_id | product_name      | aisle_id | department_id
    # 123        | 2 kg bag of flour | 1        | 3


class Orders(Base):
    __tablename__ = "orders"
    order_id = sa.Column(sa.Integer, primary_key=True)
    user_id = sa.Column(sa.Integer)
    eval_set = sa.Column(sa.String)
    order_number = sa.Column(sa.Integer)
    order_dow = sa.Column(sa.Integer)
    order_hour_of_day = sa.Column(sa.Integer)
    days_since_prior_order = sa.Column(sa.Integer)


class ProductsOrdered(Base):
    __tablename__ = "products_ordered"
    order_id = sa.Column(sa.Integer, primary_key=True)
    product_id = sa.Column(sa.Integer, primary_key=True)
    add_to_cart_order = sa.Column(sa.Integer)
    reordered = sa.Column(sa.Boolean)
    # order_id | product_id | add_to_cart_order | reordered

### A SQLalchemy engine

In [3]:
# Create a SQL alchemy engine and add table information to the engine
fn = "/home/chase/Dropbox/instacart/instacart.db"
eng = sa.create_engine(f"sqlite:///{fn}")  # postgresql:{user}:{password}@host...
Base.metadata.create_all(eng)

Session = sa.orm.sessionmaker(bind=eng)

### Dump data into database

We're going to postpone discussing what happens here for now...

We are taking the csv files that we previously saw and populating a SQLite database, but we have already constructed and included this file for you

In [ ]:
# %%time
# # Uncomment if the data needs to be fixed or updated
# # Create a SQL alchemy engine and add table information to the engine
# os.remove("/home/chase/Dropbox/instacart/instacart.db")
# Base.metadata.create_all(eng)

# Session = sa.orm.sessionmaker(bind=eng)

# # Create table -> filename pairs
# table_to_file = [
#     (Aisles, "/home/chase/Dropbox/instacart/aisles.parquet"),
#     (Departments, "/home/chase/Dropbox/instacart/departments.parquet"),
#     (Products, "/home/chase/Dropbox/instacart/products.parquet"),
#     (Orders, "/home/chase/Dropbox/instacart/orders.parquet"),
#     (ProductsOrdered,  "/home/chase/Dropbox/instacart/order_products_all.parquet"),
# ]

# session = Session()
# # Delete any data from previous inserts
# for (_t, _csv) in table_to_file:
#     session.execute(_t.__table__.delete())
#     session.commit()

# # Insert data
# for (_t, _f) in table_to_file:
#     # Read parquet file and put into the list of dictionaries
#     _rows = pd.read_parquet(_f).to_sql(
#         _t.__tablename__, eng, if_exists="append", index=False
#     )


## Reading data from a SQL database

Unless you end up becoming a data engineer, you will spend most of your time interacting with an already created database that others manage...

Because of this, we will spend most of our time focused on reading data from a database

### SQL Read Commands

We will run the raw SQL commands into the SQLAlchemy engine, but you could interact with the engine using SQLAlchemy

**Note**: It is good practice to capitalize the SQL keywords -- For example, rather than write `select` or `from`, you should write `SELECT` and `FROM`

In [4]:
def run_query(eng, query, str_length=30):
    with eng.connect() as conn:
        result = conn.execute(query)
        cols = result.keys()
        vals = result.fetchmany(5)

        fmter = ("{" + f":<{str_length}" + "}") * len(cols)
        print(fmter.format(*cols))
        for _vals in vals:
            _pvals = map(lambda x: str(x)[:str_length], _vals)
            print(fmter.format(*_pvals))

In [37]:
str_length = 5
("{", f":<{str_length}", "}") * 3

('{', ':<5', '}', '{', ':<5', '}', '{', ':<5', '}')

#### SELECT/FROM

The most fundamental read command in SQL combines the `SELECT` statement with the `FROM` statement.

* `SELECT` specifies what data to read (and what to call it)
* `FROM` specifies where that data can be read from

**Select all columns from a single table**

In [5]:
query = """
        SELECT *
        FROM products
        """

run_query(eng, query)


product_id                    product_name                  aisle_id                      department_id                 
1                             Chocolate Sandwich Cookies    61                            19                            
2                             All-Seasons Salt              104                           13                            
3                             Robust Golden Unsweetened Oolo94                            7                             
4                             Smart Ones Classic Favorites M38                            1                             
5                             Green Chile Anytime Sauce     5                             13                            


**Select certain columns**

In [6]:
query = """
        SELECT product_id, aisle_id, department_id
        FROM products
        """

run_query(eng, query)


product_id                    aisle_id                      department_id                 
1                             61                            19                            
2                             104                           13                            
3                             94                            7                             
4                             38                            1                             
5                             5                             13                            


**Select and rename certain columns**

In [7]:
query = """
        SELECT product_id AS pid, aisle_id AS aid, department_id AS did
        FROM products
        """

run_query(eng, query)


pid                           aid                           did                           
1                             61                            19                            
2                             104                           13                            
3                             94                            7                             
4                             38                            1                             
5                             5                             13                            


**Reference table using abbreviation**

In [8]:
query = """
        SELECT p.product_id AS pid, p.aisle_id, p.department_id
        FROM products p
        """

run_query(eng, query)


pid                           aisle_id                      department_id                 
1                             61                            19                            
2                             104                           13                            
3                             94                            7                             
4                             38                            1                             
5                             5                             13                            


**Select functions of columns**

In [9]:
query = """
        SELECT product_id AS pid, aisle_id, department_id, aisle_id + department_id AS a_d_id
        FROM products p
        """

run_query(eng, query)


pid                           aisle_id                      department_id                 a_d_id                        
1                             61                            19                            80                            
2                             104                           13                            117                           
3                             94                            7                             101                           
4                             38                            1                             39                            
5                             5                             13                            18                            


#### JOIN

SQL is a relational database which means that

1. We will typically store data in multiple tables
2. We'd like to be able to combine and manipulate data from multiple tables

`JOIN` allows us bring together two (or more) datasets into a single query

**Select all columns from two tables**

In [13]:
query = """
        SELECT *
        FROM products p
        JOIN aisles a ON (p.aisle_id=a.aisle_id)
        """

run_query(eng, query, 18)


product_id        product_name      aisle_id          department_id     aisle_id          aisle             
1                 Chocolate Sandwich61                19                61                cookies cakes     
2                 All-Seasons Salt  104               13                104               spices seasonings 
3                 Robust Golden Unsw94                7                 94                tea               
4                 Smart Ones Classic38                1                 38                frozen meals      
5                 Green Chile Anytim5                 13                5                 marinades meat pre


**Select subset of columns from each table**

In [14]:
query = """
        SELECT p.product_name, p.aisle_id, p.department_id, a.aisle
        FROM products p
        JOIN aisles a ON (p.aisle_id=a.aisle_id)
        """

run_query(eng, query, 30)


product_name                  aisle_id                      department_id                 aisle                         
Chocolate Sandwich Cookies    61                            19                            cookies cakes                 
All-Seasons Salt              104                           13                            spices seasonings             
Robust Golden Unsweetened Oolo94                            7                             tea                           
Smart Ones Classic Favorites M38                            1                             frozen meals                  
Green Chile Anytime Sauce     5                             13                            marinades meat preparation    


**Select data with different joins**

The merges that we've done using pandas use the same notation as SQL joins:

- `LEFT`: Use values from the left table to merge datasets
- `RIGHT`: Use values from the right table to merge datasets
- `INNER`: Only keep values contained in both the left and right datasets
- `OUTER`: Keep all values contained in either the left or right dataset.

In [16]:
query = """
        SELECT p.product_name, p.aisle_id, p.department_id, a.aisle
        FROM products p
        LEFT JOIN aisles a ON (p.aisle_id=a.aisle_id)
        """

# In this case they're all the same because there is no
# missing data...
run_query(eng, query, 30)


product_name                  aisle_id                      department_id                 aisle                         
Chocolate Sandwich Cookies    61                            19                            cookies cakes                 
All-Seasons Salt              104                           13                            spices seasonings             
Robust Golden Unsweetened Oolo94                            7                             tea                           
Smart Ones Classic Favorites M38                            1                             frozen meals                  
Green Chile Anytime Sauce     5                             13                            marinades meat preparation    


**Select data with multiple joins**

We don't have to restrict ourselves to only combining two datasets -- We can combine as many as we'd like!

In [17]:
query = """
        SELECT p.product_name, a.aisle, d.department
        FROM products p
        LEFT JOIN aisles a ON (p.aisle_id=a.aisle_id)
        LEFT JOIN departments d ON (p.department_id=d.department_id)
        """

# In this case they're all the same because there is no
# missing data...
run_query(eng, query, 30)


product_name                  aisle                         department                    
Chocolate Sandwich Cookies    cookies cakes                 snacks                        
All-Seasons Salt              spices seasonings             pantry                        
Robust Golden Unsweetened Oolotea                           beverages                     
Smart Ones Classic Favorites Mfrozen meals                  frozen                        
Green Chile Anytime Sauce     marinades meat preparation    pantry                        


#### WHERE

We are often interested in working with subsets of the data rather than selecting all of the rows.

SQL allows us to specify certain conditions to restrict the set of observations that are returned using the `WHERE` clause.

**Retrieve certain groups** (compare  strings)

In [18]:
query = """
        SELECT p.product_name, a.aisle, d.department
        FROM products p
        LEFT JOIN aisles a ON (p.aisle_id=a.aisle_id)
        LEFT JOIN departments d ON (p.department_id=d.department_id)
        WHERE d.department = 'snacks'
        """

run_query(eng, query, 30)


product_name                  aisle                         department                    
Chocolate Sandwich Cookies    cookies cakes                 snacks                        
Mint Chocolate Flavored Syrup ice cream toppings            snacks                        
Salted Caramel Lean Protein & energy granola bars           snacks                        
Nacho Cheese White Bean Chips chips pretzels                snacks                        
Organic Sourdough Einkorn Craccrackers                      snacks                        


**Retrieve certain groups** (compare numbers)

In [19]:
query = """
        SELECT p.product_name, a.aisle, d.department, a.aisle_id
        FROM products p
        LEFT JOIN aisles a ON (p.aisle_id=a.aisle_id)
        LEFT JOIN departments d ON (p.department_id=d.department_id)
        WHERE a.aisle_id > 132
        """

run_query(eng, query, 30)


product_name                  aisle                         department                    aisle_id                      
Mirabelle Brut Rose           specialty wines champagnes    alcohol                       134                           
Brut Rosé                     specialty wines champagnes    alcohol                       134                           
Ibuprofen Liquid Gels         muscles joints pain relief    personal care                 133                           
Aspirin 325 Mg                muscles joints pain relief    personal care                 133                           
Migraine Caplets              muscles joints pain relief    personal care                 133                           


**Multiple conditions**

We use `AND` and `OR` to specify the boolean condition

In [21]:
query = """
        SELECT p.product_name, a.aisle_id, d.department_id
        FROM products p
        LEFT JOIN aisles a ON (p.aisle_id=a.aisle_id)
        LEFT JOIN departments d ON (p.department_id=d.department_id)
        WHERE a.aisle_id > 100 OR d.department_id<10
        """

run_query(eng, query, 30)


product_name                  aisle_id                      department_id                 
All-Seasons Salt              104                           13                            
Robust Golden Unsweetened Oolo94                            7                             
Smart Ones Classic Favorites M38                            1                             
Pure Coconut Water With Orange98                            7                             
Cut Russet Potatoes Steam N' M116                           1                             


**Retrieve the most recent data** (compare datetime)

Imagine we had a table that contained quarterly sales

| dt | store_id | sales |
| ---- | ---- | ---- |
| 2020-03-31 | 1 | 100 |
| 2020-06-30 | 1 | 200 |
| 2020-09-30 | 1 | 300 |
| 2020-12-31 | 1 | 400 |
| 2020-03-31 | 2 | 1000 |
| 2020-06-30 | 2 | 2000 |
| 2020-09-30 | 2 | 3000 |
| 2020-12-31 | 2 | 4000 |

If we wanted to select only the observations from quarter 1, we could write

```sql
SELECT *
FROM sales
WHERE dt<'2020-04-01'
```

| dt | store_id | sales |
| ---- | ---- | ---- |
| 2020-03-31 | 1 | 100 |
| 2020-03-31 | 2 | 1000 |

If we wanted to select observations from Q3 and Q4, we could write

```sql
SELECT *
FROM sales
WHERE dt>'2020-06-31'
```

| dt | store_id | sales |
| ---- | ---- | ---- |
| 2020-09-30 | 1 | 300 |
| 2020-12-31 | 1 | 400 |
| 2020-09-30 | 2 | 3000 |
| 2020-12-31 | 2 | 4000 |

#### GROUP BY

The `GROUP BY` argument allows us to aggregate certain groups of values (much like the pandas `groupby` method).

When you perform a `GROUP BY`, any column that is not an element of the "group" must have a reduction function applied to it

**Group by single column**

In [24]:
query = """
        SELECT order_dow, COUNT(user_id) AS norder
        FROM orders o
        WHERE order_dow > 4
        GROUP BY order_dow
        """

run_query(eng, query, 15)


order_dow      norder         
5              453368         
6              448761         


**Group by multiple columns**

In [25]:
query = """
        SELECT user_id, order_dow, COUNT(order_id) AS norder
        FROM orders o
        GROUP BY user_id, order_dow
        """

run_query(eng, query, 15)


user_id        order_dow      norder         
1              1              3              
1              2              2              
1              3              2              
1              4              4              
2              1              6              


**Aggregate multiple columns**

In [26]:
query = """
        SELECT user_id, order_dow,
               COUNT(order_id) AS norder,
               AVG(days_since_prior_order) AS avg_days_since_order
        FROM orders o
        GROUP BY user_id, order_dow
        """

run_query(eng, query, 15)


user_id        order_dow      norder         avg_days_since_order
1              1              3              11.333333333333
1              2              2              19.0           
1              3              2              18.0           
1              4              4              25.25          
2              1              6              20.666666666666


#### ORDER BY

`ORDER BY` allows us to sort the output of a query

**Order by single column**

In [27]:
query = """
        SELECT order_id, user_id, order_number, days_since_prior_order
        FROM orders o
        ORDER BY user_id
        """

run_query(eng, query, 15)


order_id       user_id        order_number   days_since_prior_order
431534         1              5              28             
473747         1              3              21             
550135         1              7              20             
1187899        1              11             14             
2254736        1              4              29             


**Order by multiple columns**

In [28]:
query = """
        SELECT order_id, user_id, order_number, days_since_prior_order
        FROM orders o
        ORDER BY user_id, order_number
        """

run_query(eng, query, 15)


order_id       user_id        order_number   days_since_prior_order
2539329        1              1              None           
2398795        1              2              15             
473747         1              3              21             
2254736        1              4              29             
431534         1              5              28             


**Order by ascending/descending**

The keywords for specifying the order of ordering are `ASC` (for ascending) and `DESC` (for descending)

In [29]:
query = """
        SELECT order_id, user_id, order_number, days_since_prior_order
        FROM orders o
        WHERE days_since_prior_order < 30
        ORDER BY days_since_prior_order DESC, user_id ASC
        """

run_query(eng, query, 15)


order_id       user_id        order_number   days_since_prior_order
2254736        1              4              29             
749250         48             7              29             
1643524        48             5              29             
2994801        59             8              29             
1019719        62             11             29             


#### LIMIT

`LIMIT` is a SQL clause that specifies the (maximum) number of rows that should be returned.

It performs the same role as the pandas dataframe `head` method -- It allows you to select the $n$ largest/smallest values or simply get a preview of your data


**Retrieve first n rows**

In [30]:
%%time

query_l10 = """
        SELECT *
        FROM orders o
        LIMIT 10
        """

_ = eng.execute(query_l10).fetchall()


CPU times: user 856 µs, sys: 147 µs, total: 1 ms
Wall time: 564 µs


In [31]:
%%time

query_all = """
        SELECT *
        FROM orders o
        """

_ = eng.execute(query_all).fetchall()


CPU times: user 3.12 s, sys: 194 ms, total: 3.32 s
Wall time: 3.37 s


### Reading with pandas

We have directly used SQLAlchemy's engine to read in data up until this point, but we can also read from the engine using pandas!

In [32]:
query = """
        SELECT order_id, user_id, order_number, days_since_prior_order
        FROM orders o
        ORDER BY days_since_prior_order DESC, user_id ASC
        """

pd.read_sql(query, eng)

,order_id,user_id,order_number,days_since_prior_order
0,2550362,1,10,30.0
1,1402090,2,11,30.0
2,1492625,2,15,30.0
3,3268552,2,13,30.0
4,329954,4,6,30.0
...,...,...,...,...
3421078,969311,206205,1,NaN
3421079,3189322,206206,1,NaN
3421080,2166133,206207,1,NaN
3421081,2227043,206208,1,NaN


## Redoing our reorder example in SQL using a `WITH` clause

`WITH` clauses allow us to define a "temporary table" that can be used in a subsequent query

In [33]:
query = """
    WITH agg_po AS (
        SELECT po.product_id,
               COUNT(po.add_to_cart_order) AS norder,
               SUM(po.reordered) AS nreorder
        FROM products_ordered po
        LEFT JOIN orders o ON po.order_id=o.order_id
        WHERE o.days_since_prior_order IS NOT NULL
        GROUP BY po.product_id
    )
    SELECT apo.product_id, apo.norder, apo.nreorder,
           (apo.nreorder*1.0 / apo.norder) AS frac_reorder,
           p.product_name, p.aisle_id, p.department_id
    FROM agg_po as apo
    LEFT JOIN products p ON apo.product_id=p.product_id
    WHERE apo.nreorder > 10
    ORDER BY frac_reorder DESC
"""

df = pd.read_sql(query, eng)

In [35]:
df.tail(10)

,product_id,norder,nreorder,frac_reorder,product_name,aisle_id,department_id
31000,37518,411,18,0.043796,Ground Cardamom,104,13
31001,29736,300,13,0.043333,Organic Light Corn Syrup,29,13
31002,1658,375,16,0.042667,"Double Superfine Mustard Powder, Original English",104,13
31003,25089,647,27,0.041731,Whole Cloves,104,13
31004,42421,1003,38,0.037886,Organic Ground Nutmeg,104,13
31005,1719,321,12,0.037383,Organic Ground Mustard Seed,104,13
31006,34592,569,18,0.031634,Organic Allspice,104,13
31007,44626,595,16,0.026891,Ground Allspice,104,13
31008,6288,646,17,0.026316,Ground Cloves,104,13
31009,28741,437,11,0.025172,Organic Ground Cloves,104,13
